In [25]:
import pandas as pd
import numpy as np
import re 
import time


import requests as rq 
import bs4 as bs4 
import json
import tqdm
import glob

In [12]:
# Abre pra mim em um dataframe, separa as chaves em colunas e coloca os valores.
# É normal que ele duplique alguns elementos, normal quando estamos coletando dados, eles nao vem limpos
# fazer sort_values para ver eles duplicados
# isso so vai nos preocupar na hora da limpeza para modelagem

df =  pd.read_json("parsed_videos.json", lines=True)

In [13]:
df

,link,title,query
0,/watch?v=w63RO45LpQw,AWS Innovate 2020 with DATA and MACHINE LEARNING,machine+learning
1,/watch?v=j22SOeyETDg,Data Mining Using R: Introduction to Data Mini...,machine+learning
2,/watch?v=q2qay2nfvg4,Using a Machine-Learning Derived Algorithm Ver...,machine+learning
3,/watch?v=p_jUJ0wncxk,Machine Learning Webcam Image classification...,machine+learning
4,/watch?v=8DI_wzTIevE,Cómo Platzi logra responder tus preguntas de C...,machine+learning
...,...,...,...
1999,/watch?v=Ow__XUvn0pA,Imputation Strategy (Japanese Language) | Yuji...,kaggle
2000,/watch?v=C7Tsfrq_g18,Tips N Tricks #3: Creating a clean inference k...,kaggle
2001,/watch?v=FT4LW0kKKos,ML Project in Hindi | Kaggle Competition: Hous...,kaggle
2002,/watch?v=Z0F7e5eidsk,LANL Earthquake Prediction | Hideki Murata (cu...,kaggle


In [17]:
pd.value_counts(df.values.flatten())

machine+learning                                                                                        733
data+science                                                                                            678
kaggle                                                                                                  593
AWS Innovate 2020 with DATA and MACHINE LEARNING                                                          6
Data Mining Using R: Introduction to Data Mining Unsupervised Techniques | Machine Learning | ExcelR      6
                                                                                                       ... 
/watch?v=xMpllQLKDZ4                                                                                      1
/watch?v=wsDKJedfqSQ                                                                                      1
Digital Evangelism:   APPLY JESUS'S METHOD THROUGH DATA SCIENCE                                           1
Kaggle - Titanic Solution [1

# Coleta

Nesse video a gente vai ver como coletamos as páginas e extraimos as informações necessarias deixando no formato quase pronto para começar a modelagem

In [18]:
df.head()

,link,title,query
0,/watch?v=w63RO45LpQw,AWS Innovate 2020 with DATA and MACHINE LEARNING,machine+learning
1,/watch?v=j22SOeyETDg,Data Mining Using R: Introduction to Data Mini...,machine+learning
2,/watch?v=q2qay2nfvg4,Using a Machine-Learning Derived Algorithm Ver...,machine+learning
3,/watch?v=p_jUJ0wncxk,Machine Learning Webcam Image classification...,machine+learning
4,/watch?v=8DI_wzTIevE,Cómo Platzi logra responder tus preguntas de C...,machine+learning


In [20]:
# Alguns desses links estão duplicados, entao vamos selecionar apenas os unicos
# valores unicos da coluna link
lista_de_links = df['link'].unique()
len(lista_de_links)

1811

In [23]:
# agora vamos coletar as paginas de video
# mesmo processo com a coleta de páginas de busca
# primeiro coletamos as paginas html, brutos, e depois de verifica que a gente ta extraindo o que a gente quer, processamos as infos, para estruturar em um dataframe

url = "https://www.youtube.com{link}"

for link in lista_de_links:
    urll = url.format(link=link)
    print(urll)
    response = rq.get(urll)
    
    # peguei o link name para colocar de nome no arquivo que to salvando nos dados brutos
    # função search pega uma expressão regular, tudo que vier depois do sinal de '=', na string do link
    # ex: '/watch?v=w63RO45LpQw', tudo depois do sinal de '='
    # entao passo a string do link e a expressão regular como primeiro argumento
    # se eu pegar  .group(0), ele pegará toda a expressão regular, e retorna a expresão com v= e tudo mais
    # com group(1), somente o que está entre parenteses.
    link_name = re.search("v=(.*)", link).group(1)
    
    #feito isso pequenos uma pasta para alocar os arquivos, os codigos fonte das paginas
    with open("./dados_brutos/videos_{}.html".format(link_name), 'w+') as output:
        output.write(response.text)
        time.sleep(2)

https://www.youtube.com/watch?v=w63RO45LpQw
https://www.youtube.com/watch?v=j22SOeyETDg
https://www.youtube.com/watch?v=q2qay2nfvg4
https://www.youtube.com/watch?v=p_jUJ0wncxk
https://www.youtube.com/watch?v=8DI_wzTIevE
https://www.youtube.com/watch?v=D1TZhyd4itI
https://www.youtube.com/watch?v=uijqgxGJFkY
https://www.youtube.com/watch?v=AcmSdlOK6gI
https://www.youtube.com/watch?v=Smo_MHlxBus
https://www.youtube.com/watch?v=l1oS9A8UaiI
https://www.youtube.com/watch?v=8HWsIUHCzxk
https://www.youtube.com/watch?v=ksAGHt8hbTI
https://www.youtube.com/watch?v=a5alOgVZWDk
https://www.youtube.com/watch?v=Hiy5xvcjaiw
https://www.youtube.com/watch?v=R9cHR6ogPP0
https://www.youtube.com/watch?v=Ft-Mrafmxl8
https://www.youtube.com/watch?v=jpvWzuC8Km4
https://www.youtube.com/watch?v=-fIqWOvtFH4
https://www.youtube.com/watch?v=H2zbb69QBXc
https://www.youtube.com/watch?v=rxsF3w4E0x8
https://www.youtube.com/watch?v=95hfZ7jHWyU
https://www.youtube.com/watch?v=hLfNdacUNRA
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=apBPEJMDmLg
https://www.youtube.com/watch?v=rc_qEQCFMOY
https://www.youtube.com/watch?v=Bo8MY4JpiXE
https://www.youtube.com/watch?v=oBZzBsoihjA
https://www.youtube.com/watch?v=zikdDOzOpxY
https://www.youtube.com/watch?v=Bi7f1JSSlh8
https://www.youtube.com/watch?v=DooxDIRAkPA
https://www.youtube.com/watch?v=c0gpgCyjTM8
https://www.youtube.com/watch?v=09hTTPYV3Jg
https://www.youtube.com/watch?v=SGSOCuByo24
https://www.youtube.com/watch?v=KNAWp2S3w94
https://www.youtube.com/watch?v=9f-GarcDY58
https://www.youtube.com/watch?v=Oh4hFHlGl-s
https://www.youtube.com/watch?v=J6XcP4JOHmk
https://www.youtube.com/watch?v=oV3ZY6tJiA0
https://www.youtube.com/watch?v=ccZ2pyr3YDw
https://www.youtube.com/watch?v=VMp6pq6_QjI
https://www.youtube.com/watch?v=7R08MPXxiFQ
https://www.youtube.com/watch?v=M3lx8GHrEsg
https://www.youtube.com/watch?v=g1VeElBAeas
https://www.youtube.com/watch?v=m8-PY1oOT-8
https://www.youtube.com/watch?v=1BneqPdEhMM
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=wrBUkpiRvCA
https://www.youtube.com/watch?v=azOmzumh0vQ
https://www.youtube.com/watch?v=ALQ_RNSRE40
https://www.youtube.com/watch?v=QK_PP_2KgGE
https://www.youtube.com/watch?v=x6f5JOPhci0
https://www.youtube.com/watch?v=vKgnxf1NFB8
https://www.youtube.com/watch?v=kyNbYCHFCSw
https://www.youtube.com/watch?v=HSDyiCuViWg
https://www.youtube.com/watch?v=qhRNvCVVJaA
https://www.youtube.com/watch?v=tGyfmzuR4d4
https://www.youtube.com/watch?v=DWsJc1xnOZo
https://www.youtube.com/watch?v=ukzFI9rgwfU
https://www.youtube.com/watch?v=VKnoyiNxflk
https://www.youtube.com/watch?v=kAL2CZ7pXm8
https://www.youtube.com/watch?v=EuBBz3bI-aA
https://www.youtube.com/watch?v=Csa5R12jYRg
https://www.youtube.com/watch?v=kE5QZ8G_78c
https://www.youtube.com/watch?v=6stDhEA0wFQ
https://www.youtube.com/watch?v=oAhe4DmbygM
https://www.youtube.com/watch?v=giF8XoPTMFg
https://www.youtube.com/watch?v=mhe5e2B9bL8
https://www.youtube.com/watch?v=oQDeVVTzVPA
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=2_Jv11VpOF4
https://www.youtube.com/watch?v=u4alGiomYP4
https://www.youtube.com/watch?v=3c0Vd5FxPks
https://www.youtube.com/watch?v=ftMq5ps503w
https://www.youtube.com/watch?v=5cFUZ03Sbhc
https://www.youtube.com/watch?v=cAICT4Al5Ow
https://www.youtube.com/watch?v=xl3yQBhI6vY
https://www.youtube.com/watch?v=xWe58WGWmlk
https://www.youtube.com/watch?v=mJeNghZXtMo
https://www.youtube.com/watch?v=QDzM8r3WgBw
https://www.youtube.com/watch?v=1L0TKZQcUtA
https://www.youtube.com/watch?v=vOppzHpvTiQ
https://www.youtube.com/watch?v=f_uwKZIAeM0
https://www.youtube.com/watch?v=u73PU6Qwl1I
https://www.youtube.com/watch?v=F_VG4LNjZZw
https://www.youtube.com/watch?v=BR9h47Jtqyw
https://www.youtube.com/watch?v=_rdINNHLYaQ
https://www.youtube.com/watch?v=-ENmRfKWjmo
https://www.youtube.com/watch?v=jAA2g9ItoAc
https://www.youtube.com/watch?v=vq2nnJ4g6N0
https://www.youtube.com/watch?v=SSu00IRRraY
https://www.youtube.com/watch?v=Aut32pR5PQA
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=kOEBEEaurHg
https://www.youtube.com/watch?v=1qbD1e15Bjk
https://www.youtube.com/watch?v=Ptg92lV4E-M
https://www.youtube.com/watch?v=WKXtTo7lqmA
https://www.youtube.com/watch?v=a8N8OsR6csA
https://www.youtube.com/watch?v=Lz9PzcERe1g
https://www.youtube.com/watch?v=MzPcyWuG3wg
https://www.youtube.com/watch?v=gwbKQaUNj5U
https://www.youtube.com/watch?v=9-2OKLZfvbE
https://www.youtube.com/watch?v=6oSzDU8kkB0
https://www.youtube.com/watch?v=dM_MzB2sCQk
https://www.youtube.com/watch?v=tij6bYk3DYI
https://www.youtube.com/watch?v=Q5JyawS8f5Q
https://www.youtube.com/watch?v=oCiRv94GMEc
https://www.youtube.com/watch?v=gMoJIH0prL4
https://www.youtube.com/watch?v=_drqJ9SFCgU
https://www.youtube.com/watch?v=rdfbcdP75KI
https://www.youtube.com/watch?v=GS2J6ed1L6I
https://www.youtube.com/watch?v=BadUWF5JMn0
https://www.youtube.com/watch?v=hH4Oy-FrPVk
https://www.youtube.com/watch?v=Qz7erR3zVUc
https://www.youtube.com/watch?v=Vfo5le26IhY
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=dPwLlJkSHLo
https://www.youtube.com/watch?v=jHX0iHh6G1Q
https://www.youtube.com/watch?v=XcLO4f1i4Yo
https://www.youtube.com/watch?v=MpLHMKTolVw
https://www.youtube.com/watch?v=yl7o-56NMJ8
https://www.youtube.com/watch?v=O8NMTHYgqMo
https://www.youtube.com/watch?v=bS5PnpHmnQw
https://www.youtube.com/watch?v=G36cw9_6L3E
https://www.youtube.com/watch?v=9YTNYT1maa4
https://www.youtube.com/watch?v=YR2WYqGMLwA
https://www.youtube.com/watch?v=oy7TOjyvzPo
https://www.youtube.com/watch?v=zgXvZzt9C-c
https://www.youtube.com/watch?v=uTq_06mtjUU
https://www.youtube.com/watch?v=wY1qnvaJgUc
https://www.youtube.com/watch?v=31ozT2hUqOw
https://www.youtube.com/watch?v=mxc4cquvRUk
https://www.youtube.com/watch?v=46RjXawJQgg
https://www.youtube.com/watch?v=0CGQvdAbNcc
https://www.youtube.com/watch?v=0qaS5JyU77E
https://www.youtube.com/watch?v=TVFpKvswngo
https://www.youtube.com/watch?v=u8gDK-riNvE
https://www.youtube.com/watch?v=4CpmB4TR2C4
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=cpbtcsGE0OA
https://www.youtube.com/watch?v=d11chG7Z-xk
https://www.youtube.com/watch?v=EBmD_mnQozQ
https://www.youtube.com/watch?v=pqBCFh7SgKA
https://www.youtube.com/watch?v=xMpllQLKDZ4
https://www.youtube.com/watch?v=ph439t-kTIE
https://www.youtube.com/watch?v=RJWo8p_-Dsk
https://www.youtube.com/watch?v=mm2A5tKVIpg
https://www.youtube.com/watch?v=4y6fUC56KPw
https://www.youtube.com/watch?v=cHGhSj_Ax-0
https://www.youtube.com/watch?v=hc233A0VKYQ
https://www.youtube.com/watch?v=sivWzd_AecU
https://www.youtube.com/watch?v=_Wk9T_G-u4o
https://www.youtube.com/watch?v=xLpl4nSo3RA
https://www.youtube.com/watch?v=qWrcgHwSG8M
https://www.youtube.com/watch?v=8LucP1wiX1g
https://www.youtube.com/watch?v=oXT8luCbuQ4
https://www.youtube.com/watch?v=CKhsZI1k9bA
https://www.youtube.com/watch?v=jG6pdz4DQu8
https://www.youtube.com/watch?v=0jQ25PSyb_E
https://www.youtube.com/watch?v=n5yKDyedUes
https://www.youtube.com/watch?v=veiLCvcLIg8
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=eGdQdNcSoSA
https://www.youtube.com/watch?v=fBdkiO7-cHg
https://www.youtube.com/watch?v=d8ZCX42ep1s
https://www.youtube.com/watch?v=0lDBMhx1eqc
https://www.youtube.com/watch?v=4pkcuI5usjI
https://www.youtube.com/watch?v=dPrfYWTrZgk
https://www.youtube.com/watch?v=LpRbOzNshNA
https://www.youtube.com/watch?v=Op3019SFYzI
https://www.youtube.com/watch?v=205j37G1cxw
https://www.youtube.com/watch?v=bQwdZWeQTq0
https://www.youtube.com/watch?v=4XxMZ18w-BU
https://www.youtube.com/watch?v=LLmVWZQR5Fg
https://www.youtube.com/watch?v=crXNFxMg5R4
https://www.youtube.com/watch?v=7RdKnACscjA
https://www.youtube.com/watch?v=NGnOY-AzDBg
https://www.youtube.com/watch?v=xP99eh6nQN0
https://www.youtube.com/watch?v=3L5Co94Lilw
https://www.youtube.com/watch?v=qb-wDJhKUJY
https://www.youtube.com/watch?v=EEyHYd7J_ls
https://www.youtube.com/watch?v=Sko8u6Ay_ec
https://www.youtube.com/watch?v=MvCn7OF7HGY
https://www.youtube.com/watch?v=CfNh8piv8-w
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=sF24xXGlOAI
https://www.youtube.com/watch?v=TNzDMOg_zsw
https://www.youtube.com/watch?v=C8v-Q5ASfUE
https://www.youtube.com/watch?v=6TBvZmg7AsA
https://www.youtube.com/watch?v=2xTQR1Zd8rU
https://www.youtube.com/watch?v=6yLXy3Jydfg
https://www.youtube.com/watch?v=V_EfZXowDdY
https://www.youtube.com/watch?v=ageh45rxyXU
https://www.youtube.com/watch?v=xoTMADzMwVU
https://www.youtube.com/watch?v=lL6nzcj6H4c
https://www.youtube.com/watch?v=se4ZM0es924
https://www.youtube.com/watch?v=LEWpRlaEJO8
https://www.youtube.com/watch?v=2QjW0LyfcBo
https://www.youtube.com/watch?v=Jn8c3oe_GWU
https://www.youtube.com/watch?v=KmhGNc7gcCM
https://www.youtube.com/watch?v=lkZET6thmLk
https://www.youtube.com/watch?v=0QNgFacxvqQ
https://www.youtube.com/watch?v=s3YkZPNKLYg
https://www.youtube.com/watch?v=BifBT4DtMEc
https://www.youtube.com/watch?v=PV25kt1QiqY
https://www.youtube.com/watch?v=VC8Jc9_lNoY
https://www.youtube.com/watch?v=tsGGpe-onZI
https://www.youtube.com/watch?v=

https://www.youtube.com/watch?v=fwNLf4t7MR8
https://www.youtube.com/watch?v=JTQO01MXpr0
https://www.youtube.com/watch?v=7389fOxgBz8
https://www.youtube.com/watch?v=iP0Fxg4oqUQ
https://www.youtube.com/watch?v=F-zanmuE_VM
https://www.youtube.com/watch?v=5ylpmjG6t8E
https://www.youtube.com/watch?v=X6orAXDIrds
https://www.youtube.com/watch?v=ElDWanLOc4c
https://www.youtube.com/watch?v=i92VI289zWw
https://www.youtube.com/watch?v=LnnkCvIiQww
https://www.youtube.com/watch?v=2BrnmoQjS4w
https://www.youtube.com/watch?v=lIACsVjMNCU
https://www.youtube.com/watch?v=msUWJPXWCUM
https://www.youtube.com/watch?v=Gp_qv317Gew
https://www.youtube.com/watch?v=UZDaCot8bWg
https://www.youtube.com/watch?v=Byf4xajgx88
https://www.youtube.com/watch?v=Mw2vdYv4ups
https://www.youtube.com/watch?v=OEDUzVH1aDI
https://www.youtube.com/watch?v=IqJZYEDkmZA
https://www.youtube.com/watch?v=mbxZ_zqHV9c
https://www.youtube.com/watch?v=B3cI_Ls_3Gk
https://www.youtube.com/watch?v=bFHRmesTCc0
https://www.youtube.com/watch?v=

Temos que conferir a pagina do codigo fonte extraida, se o html tem as infos que eu quero.

ISSO É ESSENCIAL. Se a pagina é dinamica e a pagina tem elementos que são gerados na hora ou sejam gerados no seu navegados, pode nao conter as infos que queremos. Se são gerados dinamicamente, pode ser que quando baixamos o codigo fonte, so tenhamos o codigo que a pagina usa para buscar as infos no banco de dados. Não teremos as infos em especifico, mas sim uma chamada para uma API e ficamos sem o que a API retorna, por isso é bom verificar se tem realmente as info que queremos.

Novamente temos aquela pagina toda feita, sem estilos e imagens.

E vemos que o video nao está disponivel, e nao vao estar mesmo, mas nesse caso a msg esta ali pq estamos no codigo fonte, e entra naquela parte de que o navegador renderiza, e ali mostra só o esqueleto da pagina.

Temos o titulo do video, botao de inscrever...numero de view, numero de likes e deslikes, data de publicação, categoria. Varias infos que podem ser uteis para fazer previsão.

Para infos que quero agora, primeiro criamos modelo, simples, basico para termos um ponto de partida, entao nao vamos pegar todas as infos possiveis nesse momento pra criar o modelo. Vamos simplesmente criar uma infraestrutura para depois adicionar coisas e tentar melhorias.





# Processamento dos dados

Da coleta das paginas do video, vou novamente fazer o processamento, só que agora é da informação que será usada no modelo.

Isso nao siginifca que já vá criar as features, as variaveis e etc. Mas queremos as infos mais proximas possivel do formato a ser usado no modelo. X,y do aprendizado supervisionado que já connhecemos.

O código e mais complexo pq é uma pagina mais complexa. Na busca a gente queria apenas pegar o link para que pudessemos pegar essa pagina depois e tirar as informações. Aqui a gente ja vai tirar varias informações da pagina.

Uma coisa que o Mario gosta de fazer é buscar todas info possiveis. É logico que dentro de um projeto qual o orçamento e o tempo, se é possivel do ponto de vista de gestão de projeto.

Se vc puder, extraia todas as info que tiverem ali, e uma das maneiras pelas quais e bom guardar os dados brutos. Não vamos usar 90% das infos aqui mas é bom ter ali caso tenhamos uma ideia e precisemos usar.

Muitas vezes a gente não sabe o que vai ser importante antes de criar o modelo de ML, entao e importante ter o maximo de fontes de info possiveis para que possamos estar sempre melhorando o modelo.

In [29]:
# Mais uma vez vamos iterar pelos arquivos que temos de dados brutos que salvamos como video e o nome do video
# e vamos abrir o arquivo, ler string e armazenar em page_html
# bs4 basicamente destrinchar toda estrutura da pagina de forma que consigamos acessar tags, fazer busca com expressões regulares
# entramos na estrutura da pagina e fica mais facil achar as informações que queremos.
# em parsed, colocamos o argumento 'html.parser', para explicitar que é um parser de um html mesmo a ser utilizado
with open("parsed_video_info.json", 'w+') as output:
    for video_file in tqdm.tqdm_notebook(sorted(glob.glob("./dados_brutos/videos*"))):
        with open(video_file, 'r+') as inp:
            page_html = inp.read()
            parsed = bs4.BeautifulSoup(page_html, 'html.parser')
            
            # Agora vamos extair informações daquele processo de INSPECIONAR a pagina
            # Ou seja, fomos nas info que queriamos, depois inspecionar no navegador e vimos as tags que tinha, para extrair que continha as info que queriamos
            # 'find_all', busca nas 'tags' alguma que tenha a expressão que a gente passar no parenteses.
            # O que muda aqui é o seguinte: eu nao vou usar diretamente a tag, no outro colocamos find_all('a')
            # Mas aqui colocamos o argumento 'attrs' que são os atributos da tag, então eu nao to necessariamente buscando uma tag especifica, mas todas as tags que tenham uma determinada class
            # Uma class que faça o match com essa expressão regular, que tenha 'watch' no valor que foi passado pra class nessa tag
            # aqui então, podemos pegar varias tags, desde que elas tenham uma class com a palavra 'watch' dentro do valor
            class_watch = parsed.find_all(attrs={"class":re.compile(r"watch")})
            # Aqui é a mesma coisa, só que vou buscar 'watch' no 'id', nao sei pq mas no YT as vezes coloca 'watch' como 'class' e em outras 'watch' e 'id'
            # deve ser uma decisão de front end do YT, mas o que importa pra nós é pegar as informações que precisamos
            id_watch = parsed.find_all(attrs={"id":re.compile(r"watch")})
            # Na variavel channel, pegamos a tag 'a' pq vai ser o link do canal que fez o upload do video
            # aqui ja to preparando o link do canal caso queria extrair infos da pagina do canal como: outros videos, se esse canal e popular, quantos inscritos tem
            # entao ja deixo preparado, nao vou usar agora, nem buscar as paginas agora, mas ja ficam extraídas
            channel = parsed.find_all("a", attrs={"href":re.compile(r"channel")})
            # ultima tag, são as que tem a palavra 'meta', que tem algumas info sobre o video tbm.
            meta = parsed.find_all("meta")
            
            
            # Dessa vez nao sera um dicionario simples de tres elementos
            # so que pra cada elemento que ele retorna nessas variaveis que criamos.
            # ou seja, pra cada elemento das listas que ele retornar com as tags que foram encontradas com os argumentos que a gente passou, vou ter que processar de uma maneira diferente
            data = dict()
            
            # No primeiro, nos elementos da class watch, ele pode ter mais de um valor na class, 
            # aqui eu quero que ele junte tudo e coloque numa string, pq vai ser o nome que vou usar para essa informação
            # o valor da classe desses elementos class watch, vai ser o nome desse atributo nos meus dados, o nome da coluna que vou ter no pandas referente a esses dados
            # esse if, vai simplesmente pular todos os elementos que tenha essa classe 'clearfix', pois ele duplica, e um argumento especifico para cada video, que cria uma coluna especifica pra cada video, ele cria uma coluna pra cada video
            # Ao final ele coloca a chave colname no dicionario e coloca o valor desse elemento com essa função strip que tira os espaços em branco do começo e fim da string
            for e in class_watch:
                colname = "_".join(e['class'])
                if "clearfix" in colname:
                    continue
                data[colname] = e.text.strip()
            
            # Mesma coisa aqui no id_watch, só que agora o nome da coluna é id. 
            for e in id_watch:
                colname = e['id']
                #if colname in output:
                #      print(colname)
                data[colname] = e.text.strip()
            
            # Aqui vamos pegar um valor dentro do elemento 'property', e.get vai procurar se existe o property, se nao existe ele retorna um Nan
            # Se tiver a coluna e não for none, ele coloca a propriedade 'content' como valor dessa coluna dos nosso dados
            for e in meta:
                colname = e.get('property')
                if colname is not None:
                    data[colname] = e['content']
            
            # Quando chegamos na lista de canais, ele achava vários links dentro da mesma pagina que ele chamava de canal
            # 
            for link_num, e in enumerate(channel):
                data['channel_link_{}'.format(link_num)] = e['href']
                
            # geralmente quando fizemos coleta de dados de paginas, o codigo fica carregado mesmo.
            # vamos ver o que tem nessas funções pra entender melhor o que está acontencendo.
            output.write("{}\n".format(json.dumps(data)))
                

# Vamos ver algumas dessas variaveis 

In [30]:
len(data) # 119 chaves, 119 colunas pelo menos, pode ser que tenha mais, pois alguns dict podem ter mais info que outros

119

### Class_watch

In [31]:
# vemos que tem o 'clearfix', nao quero isso pq ele cria uma coluna pra cada video
class_watch[:3]

[<div class="watch video-zz05VaLcgsw clearfix" id="page"><div class="yt-scrollbar" id="guide"> <div class="appbar-menu appbar-guide-menu-layout appbar-guide-clickable-ancestor" id="appbar-guide-menu">
 <div id="guide-container">
 <div class="guide-module-content guide-module-loading">
 <p class="yt-spinner">
 <span class="yt-spinner-img yt-sprite" title="Carregando ícone"></span>
 <span class="yt-spinner-message">
 Carregando...
     </span>
 </p>
 </div>
 </div>
 </div>
 </div><div class="alerts-wrapper"><div class="content-alignment" id="alerts"> <div class="yt-alert yt-alert-default yt-alert-warn" id="old-browser-alert"> <div class="yt-alert-icon">
 <span class="icon master-sprite yt-sprite"></span>
 </div>
 <div class="yt-alert-content" role="alert"> <div class="yt-alert-message" tabindex="0">
             Não daremos mais suporte a este navegador em breve. Para melhorar sua experiência, <a href="/supported_browsers">atualize seu navegador</a>.
 
     </div>
 </div><div class="yt-a

In [32]:
# entendendo o que o loop ta fazendo, apenas no primeiro elemento
# ele pega os elementos em class, "watch video-zz05VaLcgsw clearfix"
class_watch[0]

<div class="watch video-zz05VaLcgsw clearfix" id="page"><div class="yt-scrollbar" id="guide"> <div class="appbar-menu appbar-guide-menu-layout appbar-guide-clickable-ancestor" id="appbar-guide-menu">
<div id="guide-container">
<div class="guide-module-content guide-module-loading">
<p class="yt-spinner">
<span class="yt-spinner-img yt-sprite" title="Carregando ícone"></span>
<span class="yt-spinner-message">
Carregando...
    </span>
</p>
</div>
</div>
</div>
</div><div class="alerts-wrapper"><div class="content-alignment" id="alerts"> <div class="yt-alert yt-alert-default yt-alert-warn" id="old-browser-alert"> <div class="yt-alert-icon">
<span class="icon master-sprite yt-sprite"></span>
</div>
<div class="yt-alert-content" role="alert"> <div class="yt-alert-message" tabindex="0">
            Não daremos mais suporte a este navegador em breve. Para melhorar sua experiência, <a href="/supported_browsers">atualize seu navegador</a>.

    </div>
</div><div class="yt-alert-buttons"></div>

In [33]:
e = class_watch[0]
e['class']

# simplesmente to mandando fazer o join desses elemento em uma string
# depois vai verificar e existencia do clearfix, nesse caso tem, entao pula esse exemplo e vai para o proximo

['watch', 'video-zz05VaLcgsw', 'clearfix']

In [35]:
e = class_watch[1]
e['class']

# o proximo elemento, nao tem 'clearfix', extraiu pq tem o watch-smal na classe
# vai fazer o .join das duas

['content-alignment', 'watch-small']

In [36]:
# juntou as strings, e vai ser o nome da coluna no pandas, ou nome da chave no json, no dicionario
# e o valor dessa chave vai ser... e.text.strip()

'_'.join(e['class'])

'content-alignment_watch-small'

In [37]:
# o valor vai ser esse negocio todo,no caso desse elemento.
# quer dizer que cada elemento que ele achou 'watch' na pagina do video, vai ganhar uma coluna desde que nao seja 'clearfix'

e.text.strip()

'Este vídeo não está disponível.\n\n    \n\n\n\n\n\nif (window.ytcsi) {window.ytcsi.tick("cfg", null, \'\');}\nvar ytplayer = ytplayer || {};ytplayer.config = {"args":{"innertube_api_version":"v1","fflags":"html5_ios4_seek_above_zero=true\\u0026player_allow_autonav_after_playlist=true\\u0026html5_video_tbd_min_kb=0\\u0026web_logging_max_batch=100\\u0026preskip_button_style_ads_backend=countdown_next_to_thumbnail\\u0026mweb_native_control_in_faux_fullscreen_shared=true\\u0026html5_av1_thresh_hcc=1080\\u0026web_network_combined_catch=true\\u0026ensure_vis_persists_in_full_screen_for_mweb=true\\u0026html5_aspect_from_adaptive_format=true\\u0026html5_probe_primary_failure_factor=4\\u0026web_player_live_monitor_env_killswitch2=true\\u0026info_cards_renderer_on_desktop=true\\u0026desktop_videowall_companion_wta_support=true\\u0026html5_sticky_reduces_discount_by=0.0\\u0026disable_legacy_desktop_remote_queue=true\\u0026html5_no_placeholder_rollbacks=true\\u0026html5_ios_force_seek_to_zero_on_

### id_watch

In [38]:
# Vamos ver agora do id watch, que acaba sendo mais simples, pq se a gente olhar , dessa vez quando o id tiver a palavra watch

e = id_watch[0]
e

# nesse caso id="watch-queue-mole"

<div class="video-mole mole-collapsed hid" id="watch-queue-mole"><div class="watch-playlist player-height" id="watch-queue"><div class="main-content"><div class="watch-queue-header"><div class="watch-queue-info"><div class="watch-queue-info-icon"><span class="tv-queue-list-icon yt-sprite"></span></div><h3 class="watch-queue-title">Fila de exibição</h3><h3 class="tv-queue-title">Fila</h3><span class="tv-queue-details"></span></div><div class="watch-queue-control-bar control-bar-button"><div class="watch-queue-mole-info"><div class="watch-queue-control-bar-icon"><span class="watch-queue-icon yt-sprite"></span></div><div class="watch-queue-title-container"><span class="watch-queue-count"></span><span class="watch-queue-title">Fila de exibição</span><span class="tv-queue-title">Fila</span></div></div> <span class="dark-overflow-action-menu">
<button aria-expanded="false" aria-haspopup="true" aria-label="Ações para a fila" class="flip control-bar-button yt-uix-button yt-uix-button-dark-over

In [ ]:
# ai tu fala, po: isso nao e informação relevante pra gente, mas não é nesse caso, nesse elemento, mas tem outros elementos que podem ter infos importantes

e.text.strip()

### meta

In [40]:
e = meta[0]
e

<meta content="What's the Best Career Advice You've Been Given? | Kaggle CareerCon 2019" name="title"/>

In [41]:
# se pegarmos o item 4, ja vemos que surge o 'property', pq eu quero isso ? pois vai ser o nome da coluna que vou usar no pandas
# então to colocando dentro do dicionario de dados desse video especifico, coloquei 
e = meta[4]
e

<meta content="YouTube" property="og:site_name"/>

In [42]:
#  então to colocando dentro do dicionario de dados desse video especifico, coloquei,a chave "og:site_name", e o valor dessa chave vai ser:
e['content']

'YouTube'

### channel

In [44]:
# pra finalizar, se a gente vÊ é essa coisa grande, e vemos que tem mais de um link
# qual é o link que a gente realmente quer, entao na duvida eu pego e armazeno todos eles.
# usamos 'enumerate' que coloca numero pra cada elemento que vamos iterar, e a forma que encontramos de identificar que esses são os links que o parser achou, entao vamos ter channel_link 0, channel_link 1... e etc
# e cada um deles vai se referir a um dos elementos que temos nessa lista
e = channel[1]
e

<a class="yt-uix-sessionlink" data-sessionlink="itct=CCwQ6TgYACITCODlk_Tq3ucCFQfYkQodSBIBEEjMhfKW2qrOns8B" data-url="http://www.youtube.com/user/kaggledotcom?sub_confirmation=1&amp;utm_medium=youtube&amp;utm_source=channel&amp;utm_campaign=yt-sub" href="http://www.youtube.com/user/kaggledotcom?sub_confirmation=1&amp;utm_medium=youtube&amp;utm_source=channel&amp;utm_campaign=yt-sub">http://www.youtube.com/user/kaggledot...</a>

PQ MÁRIO ESCOLHEU ESPECIFICAMENTE ESSAS 4 VÁRIAVEIS ?